In [1]:
# This cell is tagged "parameters" to enable running this notebook from the command line with papermill
# Example usage:
# papermill -p USER aubreymoore -p USER_FULL_NAME 'Aubrey Moore' inat_labels.ipynb aubreymoore.ipynb

INSECTS_OF_MICRONESIA_CSV = 'observations-203838.csv'

FONT_SIZE = 3.5    # points
FONT = 'helvetica'
YMIN = 1           # inches
LABEL_WIDTH = 1.0  # inches
LABEL_HEIGHT = 1.0 # inches
XMIN = 0.5         # inches

NROWS = 9
NCOLS = 7

USER = 'haanibettis'
USER_FULL_NAME = 'Haani Bettis'

DESC_MAX_CHARS = 100
PRINT = True

In [2]:
# Parameters
INSECTS_OF_MICRONESIA_CSV = "/home/aubrey/Desktop/inat_labels/observations-203843.csv"
USER = "rachelm23"
USER_FULL_NAME = "Rachel Mungon"
PRINT = "TRUE"


In [3]:
import subprocess
import sqlite3
from fpdf import FPDF
import qrcode
from datetime import datetime
import os
import pandas as pd

In [4]:
def add_taxonomy(obs_dict):
    ancestry = obs_dict['ancestry'].split('/')
    ancestry.append(obs_dict['taxon'])
    for t in ancestry:
        sql = f'SELECT rank, name FROM taxons WHERE id == {t}'
        cursor = conn.execute(sql)
        for row in cursor:
            obs_dict[row[0]] = row[1]

In [5]:
def get_label_data():
    data_list = []
    sql = """
    SELECT observations.id, name, rank, observed_on, taxon, place_guess, longitude, latitude, description, project_ids, taxons.ancestry
    FROM observations, taxons
    WHERE 
       taxons.id == taxon
       AND project_ids LIKE '%1627%'
       AND observed_on > '2021-08-01';
    """
    cursor = conn.execute(sql)
    for r in cursor:
        obs_dict = dict(r)
        add_taxonomy(obs_dict)
        for key in ['ancestry', 'rank', 'name', 'project_ids', 'taxon']:
            del obs_dict[key]
        data_list.append(obs_dict)        
    return data_list

# get_label_data()

In [6]:
class PDF(FPDF):
    
    def add_label(self, x, y, obs):
        self.set_xy(x, y)
        self.multi_cell(txt=f'iNAT {obs["id"]}', w=LABEL_WIDTH, ln=2, align='L')
        #print(f'1:{self.get_x()} {self.get_y()}')
        self.multi_cell(txt=obs['place_guess'], w=LABEL_WIDTH, ln=2, align='L')
        #print(f'2:{self.get_x()} {self.get_y()}')
        
        lat = float(obs['latitude'])
        lat = round(lat,6)
        if (lat > 0):
            lat_suffix = 'N'
        else:
            lat_suffix = 'S'       
        
        lon = float(obs['longitude'])
        lon = round(lon, 6)
        if (lon > 0):
            lon_suffix = 'E'
        else:
            lon_suffix = 'W'
        txt = f'{lat}{lat_suffix} {lon}{lon_suffix}' 
        self.multi_cell(txt=txt, w=LABEL_WIDTH, ln=2, align='L')

        txt = obs['description']
        if isinstance(txt, str) and (len(txt) > 1):
            if len(txt) > DESC_MAX_CHARS:
                txt = f'{txt[0:DESC_MAX_CHARS]}...'
            self.multi_cell(txt=txt, w=LABEL_WIDTH, ln=2, align='L')
       
        txt = f'Col. {USER_FULL_NAME} {obs["observed_on"]}'
        self.multi_cell(txt=txt, w=LABEL_WIDTH, ln=2, align='L')
        
        self.multi_cell(txt='---', w=LABEL_WIDTH, ln=2, align='L')
        
        self.multi_cell(txt=f'iNAT {obs["id"]}', w=LABEL_WIDTH, ln=2, align='L')
        
        if obs.get('taxon_order_name'):
            txt = obs['taxon_order_name']
            if isinstance(txt, str):
                self.multi_cell(txt=txt.upper(), w=LABEL_WIDTH, ln=2, align='L')
            
        if obs.get('taxon_family_name'):
            txt = obs["taxon_family_name"]  
            if isinstance(txt, str):
                txt = f'   {txt.upper()}'
                self.multi_cell(txt=txt, w=LABEL_WIDTH, ln=2, align='L')
            
        if obs.get('taxon_species_name'):   # species binomial available
            txt = obs.get('taxon_species_name')
            if isinstance(txt, str):
                txt = f'     {txt}'
                self.set_font(style="I")
                self.multi_cell(txt=txt, w=LABEL_WIDTH, ln=2, align='L')
                self.set_font(style="")
        else:
            if obs.get('taxon_genus_name'):   # only genus available
                txt = obs['taxon_genus_name']
                if isinstance(txt, str):
                    txt = f'     {obs["genus"]}'
                    self.set_font(style="I")
                    self.multi_cell(txt=txt, w=LABEL_WIDTH, ln=2, align='L')
                    self.set_font(style="")

        # Check on this.
        txt = f'Det. {USER_FULL_NAME} {obs["observed_on"]}'
        self.multi_cell(txt=txt, w=LABEL_WIDTH, ln=2, align='L')
        
        # barcode
        
        self.multi_cell(txt='---', w=LABEL_WIDTH, ln=2, align='L')
        self.multi_cell(txt=f'iNAT {obs["id"]}', w=LABEL_WIDTH, ln=2, align='L')
        url = f"https://www.inaturalist.org/observations/{obs['id']}"
        img = qrcode.make(url)
        self.image(img.get_image(), w=0.25)

In [7]:
def generate_labels():

    pdf = PDF(orientation="P", unit="in", format="Letter")
    pdf.set_doc_option('core_fonts_encoding', 'utf-8') #NEW
    pdf.set_font(family=FONT, size=FONT_SIZE)
    
    n = len(data_list)
    i = 0
    pages = 1 + n // (NROWS * NCOLS)
    for p in range(pages):
        print(f'Adding page {p}')
        pdf.add_page()
        txt=f'{USER_FULL_NAME}          Page {p+1} of {pages}          {datetime.now()}'
        pdf.set_font(size=14)
        pdf.multi_cell(txt=txt, w = 6)
        pdf.set_font(size=FONT_SIZE)
        r = 0
        c = 0
        for r in range(NROWS):
            y = YMIN + r * LABEL_HEIGHT
            for c in range(NCOLS):
                x = XMIN + c * LABEL_WIDTH
#                 print(f'{i} {p} {r} {c} {x} {y}')
#                 print(data_list[i])
                pdf.add_label(x, y, data_list[i])
                i += 1
                if i == n:
                    print(f"writing {pdf_filename}")
                    pdf.output(pdf_filename)
                    return

# generate_labels()

In [8]:
# MAIN

# Make sure there is an output directory

if not os.path.exists('output'):
    os.makedirs('output')

df = pd.read_csv(INSECTS_OF_MICRONESIA_CSV)
df = df[df['user_login']==USER]
data_list = df.to_dict(orient='records')

# Generate labels

pdf_filename = f"output/{USER_FULL_NAME.replace(' ', '_')}.pdf"
generate_labels()

if PRINT:
    subprocess.run(['lpr', '-P', 'HP-LaserJet-P1006', pdf_filename])

print('FINISHED')

Adding page 0


writing output/Rachel_Mungon.pdf
FINISHED
